In [ ]:
from pathlib import Path
import numpy as np
from scipy.optimize import curve_fit

from astropy.table import Table, vstack
from astropy.coordinates import SkyCoord

%matplotlib inline
import matplotlib.pyplot as plt

from stellarphot.utils.magnitude_transforms import (
    calibrated_from_instrumental,
    opts_to_str,
    calc_residual,
    transform_to_catalog,
)

from stellarphot import PhotometryData, vsx_vizier

## File names

In [ ]:
input_file_name = "photometry.ecsv"

In [ ]:
input_photometry_file = Path(input_file_name)
output_photometry_file = input_photometry_file.parent / (input_photometry_file.stem + "-transformed" + input_photometry_file.suffix)
output_photometry_file

# Parameters

The magnitudes in each image are fit using this model:

$$
r_{p, c} = a r_{p, inst} + b r_{p, inst}^2 + c (B_c - V_c) + d (B_c - V_c)^2 + z
$$

The parameters in the cell below set the range of values the fit is constrained to. The way to fix a parameter is to give it a very, very small range for the constraint.

More specifically, each of the fit values is subject to these constraints:

+ $1 - a_{delta} < a < 1 + a_{delta}$
+ $b_{min} < b < -b_{min}$
+ $c_{min} < c < -c_{min}$
+ $d_{min} < d < -d_{min}$
+ The range for the zero point is $18 < z < 22$.


`output_dir` is where the PNG and FITS files generated by this notebook are stored. `run_name` is a descriptive name for the settings you have chosen that gets included in the output file names.

### *Recommendation:*

+ Keep $b$ essentially fixed. 
+ Fixing $d$ is ok for now too, I think.

In both cases, setting a min of `1e-6` or something should do the trick.


In [ ]:
a_delta = 0.5
b_delta = 1e-6
c_delta = 0.2
d_delta = 1e-6

catalog = "apass_dr9"   # or "refcat2"

cat_color_colums = dict(
    B=("B", "V"),
    V=("B", "V"),
    SG=("SG", "SR"),
    SR=("SR", "SI"),
    SI=("SR", "SI"),
    I=("R", "I"),
    R=("R", "I"),
)



In [ ]:
all_mags = PhotometryData.read(input_photometry_file)
all_mags.add_bjd_col()

## Do the transforms, one filter at a time

In [ ]:
output_table = []
filter_groups = all_mags.group_by("passband")

for key, group in zip(filter_groups.groups.keys, filter_groups.groups):
    # The key is a table column, not a value...
    k = key[0]
    print(f"Transforming band {k}")
    by_bjd = group.group_by("file")

    transform_to_catalog(
        by_bjd,
        k,
        obs_error_column="mag_error",
        zero_point_range=[12, 25],
        c_delta=c_delta, b_delta=b_delta, d_delta=d_delta, a_delta=a_delta,
        cat_name=catalog,
        cat_filter=k,
        cat_color=cat_color_colums[k],
        in_place=True,
    )
    output_table.append(by_bjd.copy())
    one_image = by_bjd[by_bjd["file"] == list(set(by_bjd["file"]))[0]]
    plt.figure()
    plt.plot(one_image["mag_inst"], one_image["mag_cat"], ".", alpha=0.4)
    #plt.plot([-10.665, -4.703], [9.59, 15.339])
    plt.xlabel(f"Instrumental magnitude {k}")
    plt.ylabel(f"Catalog mag {k}")
    plt.grid()
output_table = vstack(output_table, join_type="outer")

In [ ]:
one_image = by_bjd[by_bjd["file"] == list(set(by_bjd["file"]))[0]]

In [ ]:
plt.figure()
plt.plot(one_image["mag_inst"], one_image["mag_cat"], ".", alpha=0.4)
#plt.plot([-10.665, -4.703], [9.59, 15.339])
plt.xlabel("Instrumental magnitude")
plt.ylabel("Catalog mag")
plt.grid()

In [ ]:
plt.figure()
plt.plot(one_image["snr"], one_image["mag_cat"] - one_image["mag_inst_cal"], ".", alpha=0.4)
plt.xlabel("SNR")
plt.ylabel("Catalog mag - feder mag")
plt.grid()

In [ ]:
plt.figure()
plt.plot(one_image["mag_inst_cal"], one_image["mag_cat"] - one_image["mag_inst_cal"], ".", alpha=0.4)
plt.xlabel("calibrated Feder mag")
plt.ylabel("Catalog mag - feder mag")
plt.grid()

In [ ]:
plt.figure()
plt.plot(one_image["mag_inst"], one_image["mag_cat"] - one_image["mag_inst_cal"], ".", alpha=0.4)
plt.xlabel("Instrumental Feder mag")
plt.ylabel("Catalog mag - feder mag")
plt.grid()

In [ ]:
plt.figure()
plt.plot(one_image["color_cat"], one_image["mag_cat"] - one_image["mag_inst_cal"], ".", alpha=0.4)
plt.xlabel("catalog color")
plt.ylabel("Catalog mag - feder mag")
plt.grid()

In [ ]:
plt.figure()
rp_only = output_table[output_table["passband"] == "SR"]
plt.plot(rp_only["bjd"].value, rp_only["z"])
plt.title("z")
plt.grid()

In [ ]:
output_table.write(output_photometry_file, overwrite=True)